In [3]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from pprint import pprint


from sklearn.pipeline import Pipeline


import numpy as np
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import GridSearchCV
from sklearn import metrics


import pandas

In [4]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

In [5]:
# categories = ["alt.atheism", "alt.atheism" ,"comp.os.ms-windows.misc" , "comp.sys.ibm.pc.hardware",
#                 "comp.sys.mac.hardware" , "comp.windows.x","misc.forsale", "rec.autos","rec.motorcycles",
#                 "rec.sport.baseball","rec.sport.hockey", "sci.crypt","sci.electronics", "sci.med","sci.space",
#                 "soc.religion.christian","talk.politics.guns" ,"talk.politics.mideast","talk.politics.misc" ,"talk.religion.misc"]

In [6]:
twenty_train = fetch_20newsgroups(subset='train',categories=categories, shuffle=True, random_state=42)

In [7]:
len(twenty_train.data)

2257

In [8]:
dir(twenty_train)

['DESCR', 'data', 'filenames', 'target', 'target_names']

In [9]:
# print(twenty_train.DESCR)

In [10]:
# twenty_train.data

In [11]:
# twenty_train.filenames

In [12]:
# twenty_train.target

In [13]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

* dataFrame

In [14]:
# data1 = pd.DataFrame(twenty_train.data,columns=['text'])
# data1['Target'] = pd.DataFrame(twenty_train.target)
# # data1.columns = ['text']

In [15]:
# data1

In [16]:
len(twenty_train.filenames)

2257

In [17]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton


In [18]:
print(twenty_train.target_names[twenty_train.target[0]])

comp.graphics


In [19]:
twenty_train.target[:10]

array([1, 1, 3, 3, 3, 3, 3, 2, 2, 2], dtype=int64)

In [20]:
for t in twenty_train.target[:10]:
    print(twenty_train.target_names[t])

comp.graphics
comp.graphics
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
sci.med
sci.med
sci.med


In [21]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35788)

In [22]:
# dir(count_vect)

In [23]:
len(count_vect.vocabulary_)

35788

In [24]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

In [25]:
print(X_train_tfidf[3])

  (0, 35780)	0.2952097830020572
  (0, 35351)	0.08554246433423428
  (0, 32270)	0.027514398126678302
  (0, 32142)	0.12262165831237365
  (0, 32138)	0.10816776858375814
  (0, 31945)	0.06350223142160838
  (0, 31077)	0.019361516438167272
  (0, 30673)	0.0652556021612127
  (0, 28877)	0.10302335237274547
  (0, 28498)	0.2952097830020572
  (0, 28425)	0.2952097830020572
  (0, 28256)	0.20007752290263073
  (0, 28104)	0.15545531085676811
  (0, 27510)	0.074337241698778
  (0, 27456)	0.11777953951863711
  (0, 27149)	0.0833202363217539
  (0, 25461)	0.27542906557889263
  (0, 24382)	0.0916429111981076
  (0, 23915)	0.02047323456491503
  (0, 23870)	0.06193238297981811
  (0, 23733)	0.058900205913078055
  (0, 23610)	0.042070557103992996
  (0, 23102)	0.29383074706041146
  (0, 22270)	0.05063446735796336
  (0, 22215)	0.04195404116840347
  :	:
  (0, 15305)	0.10655337419164386
  (0, 14887)	0.019361516438167272
  (0, 14842)	0.11186688769093728
  (0, 14601)	0.025423574335665014
  (0, 14285)	0.10734374990958656
  (0, 

In [26]:
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [27]:
docs_new = ['love', 'OpenGL on the GPU is fast']

In [28]:
X_new_counts = count_vect.transform(docs_new)

In [29]:
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [30]:
predicted = clf.predict(X_new_tfidf)

In [31]:
for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


# Construire un pipeline

In [32]:
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

In [33]:
text_clf.fit(twenty_train.data, twenty_train.target)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

## Évaluation des performances sur l'ensemble de tests

In [34]:
twenty_test = fetch_20newsgroups(subset='test',
    categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.8348868175765646

In [35]:
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)),
])

text_clf.fit(twenty_train.data, twenty_train.target)

predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)


0.9101198402130493

In [36]:
print(metrics.classification_report(twenty_test.target, predicted,
    target_names=twenty_test.target_names))


                        precision    recall  f1-score   support

           alt.atheism       0.95      0.80      0.87       319
         comp.graphics       0.87      0.98      0.92       389
               sci.med       0.94      0.89      0.91       396
soc.religion.christian       0.90      0.95      0.93       398

              accuracy                           0.91      1502
             macro avg       0.91      0.91      0.91      1502
          weighted avg       0.91      0.91      0.91      1502



In [37]:
metrics.confusion_matrix(twenty_test.target, predicted)

array([[256,  11,  16,  36],
       [  4, 380,   3,   2],
       [  5,  35, 353,   3],
       [  5,  11,   4, 378]], dtype=int64)

# Réglage des paramètres à l'aide de la recherche dans la grille

In [38]:
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    'clf__alpha': (1e-2, 1e-3),
}

In [39]:
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)

In [40]:
gs_clf = gs_clf.fit(twenty_train.data[:400], twenty_train.target[:400])

In [41]:
twenty_train.target_names[gs_clf.predict(['God is love'])[0]]

'soc.religion.christian'

In [42]:
gs_clf.best_score_

for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

clf__alpha: 0.001
tfidf__use_idf: True
vect__ngram_range: (1, 1)
